In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
import pickle
with open('/content/drive/My Drive/Prepared_Data.pkl', 'rb') as f:
    df_prep = pickle.load(f)

In [ ]:
df_prep

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE,DAYS_NEXT_ADMIT,CATEGORY,TEXT,OUTPUT_LABEL
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,Admission Date: [**2101-10-20**] Discharg...,0
2,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,Admission Date: [**2191-3-16**] Discharge...,0
4,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,NaT,NaN,NaN,Discharge summary,Admission Date: [**2175-5-30**] Dischar...,0
7,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,NaT,NaN,NaN,Discharge summary,"Name: [**Known lastname 10050**], [**Known fi...",0
9,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,Admission Date: [**2178-4-16**] ...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
58971,99985,176670,2181-01-27 02:47:00,2181-02-12 17:05:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,Admission Date: [**2181-1-27**] ...,0
58972,99991,151118,2184-12-24 08:30:00,2185-01-05 12:15:00,NaT,ELECTIVE,NaT,NaN,NaN,Discharge summary,Admission Date: [**2184-12-24**] ...,0
58973,99992,197084,2144-07-25 18:03:00,2144-07-28 17:56:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,Admission Date: [**2144-7-25**] ...,0
58974,99995,137810,2147-02-08 08:00:00,2147-02-11 13:15:00,NaT,ELECTIVE,NaT,NaN,NaN,Discharge summary,Admission Date: [**2147-2-8**] D...,0


In [ ]:
#row_not_death = df_prep.DEATHTIME.isnull()
 # exclude died patients

In [ ]:
#df_prep = df_prep.loc[row_not_death].copy()

In [ ]:
#df_prep

In [ ]:
#df_prep = df_prep.sample(n = len(df_prep), random_state = 42)

In [ ]:
#df_prep = df_prep.reset_index(drop = True)

Save 20% of the data for validation

In [ ]:
df_valid = df_prep.sample(frac = 0.2, random_state = 42)   # use shuffled .sample

In [ ]:
df_valid.to_pickle('/content/drive/My Drive/df_valid_LSTM.pkl')

In [ ]:
df_train_all = df_prep.drop(df_valid.index)

In [ ]:
row_pos = df_train_all.OUTPUT_LABEL == 1
row_pos

1        False
2        False
4        False
7        False
10       False
         ...  
58971    False
58972    False
58973    False
58974    False
58975    False
Name: OUTPUT_LABEL, Length: 40890, dtype: bool

In [ ]:
~ row_pos

1        True
2        True
4        True
7        True
10       True
         ... 
58971    True
58972    True
58973    True
58974    True
58975    True
Name: OUTPUT_LABEL, Length: 40890, dtype: bool

In [ ]:
df_train_pos = df_train_all.loc[row_pos]
df_train_neg = df_train_all.loc[~row_pos]
len(df_train_pos)

2416

In [ ]:
len(df_train_neg)

38474

Downsampling and shuffle again to mix pos and neg

In [ ]:
df_train = pd.concat([df_train_pos,df_train_neg.sample(n = len(df_train_pos),random_state = 42)],axis=0)

In [ ]:
df_train = df_train.sample(n=len(df_train),random_state = 42).reset_index(drop = True)

In [ ]:
len(df_train)

4832

In [ ]:
df_train.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE,DAYS_NEXT_ADMIT,CATEGORY,TEXT,OUTPUT_LABEL
0,15453,168028,2110-12-07 23:49:00,2110-12-13 10:56:00,NaT,EMERGENCY,2110-12-14 22:44:00,EMERGENCY,1.491667,Discharge summary,Admission Date: [**2110-12-7**] Dischar...,1
1,31972,152547,2163-12-22 17:23:00,2163-12-23 04:08:00,2163-12-23 04:08:00,EMERGENCY,2163-12-23 04:08:00,EMERGENCY,0.000000,Discharge summary,Admission Date: [**2163-12-22**] ...,1
2,20928,113718,2114-02-23 10:30:00,2114-03-15 14:00:00,NaT,ELECTIVE,NaT,NaN,NaN,Discharge summary,Admission Date: [**2114-2-23**] ...,0
3,98905,110019,2184-09-24 07:15:00,2184-09-24 20:22:00,NaT,ELECTIVE,NaT,NaN,NaN,NaN,NaN,0
4,19819,151971,2173-02-11 20:56:00,2173-02-19 15:25:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,Admission Date: [**2173-2-11**] ...,0


In [ ]:
df_train['TEXT'][11]

"Admission Date:  [**2188-5-27**]     Discharge Date:  [**2188-6-4**]\n\nDate of Birth:   [**2112-8-21**]     Sex:  F\n\nService:\n\nPRESENT ILLNESS:\n1.  Acute renal failure.\n2.  Resolving urinary tract infection, status post klebsiella\nurosepsis.\n3.  Prolonged ventilator dependence.\n4.  Status post [**Last Name (un) 3696**] syndrome.\n5.  Diabetes mellitus.\n\nHISTORY OF PRESENT ILLNESS:  Mrs. [**Known lastname 20728**] is a 75-year-old\nfemale with a history of insulin dependent diabetes,\nhypertension, morbid obesity, coronary artery disease, status\npost three vessel coronary artery bypass graft in [**2188-3-25**], paroxysmal atrial fibrillation, congestive heart\nfailure with normal ejection fraction who was transferred\nfrom [**Hospital1 **] for evaluation and therapy of acute\nrenal failure, [**Last Name (un) 3696**] syndrome and persistent ventilator\ndependence with metabolic acidosis and resolving Klebsiella\nurosepsis.\n\nPatient initially presented to [**Hospital1 188*

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#all_stopwords

In [ ]:
kept = ['not','very',"wouldn't", 'must',"won't","shouldn't","haven't","hasn't","hadn't","haven't","couldn't"]
new_stop = list(set(all_stopwords) - set(kept))
newStopWords = ['admission','discharge','date','name','lastname','firstname','identifier','birth','sex','service']
new_stop.extend(newStopWords)

In [ ]:
#new_stop

In [ ]:
import re
def clean_text (x):
  x = re.sub(r'[^A-Za-z]', ' ' , x).lower()
  x = x.split()
  new_list = []
  for word in x:
    if word not in new_stop:
      new_list.append(word)
  return " ".join(new_list)

In [57]:
# clean_text(df_train['TEXT'][0])
# clean_text(df_valid['TEXT'][0])

'history present illness patient year old male history hepatitis c virus hepatocellular cancer status post chemoembo presented hospital e hematemesis since one episode hematemesis two episodes well hematemesis hematochezia hospital e patient noted hematocrit transfused two units packed red blood cells noted coagulopathy inr given ffp well vitamin k patient denies past history varices upper gi bleed denies recent alcohol drug use feeling nauseated began vomiting continued hematemesis three days transferred hospital definitive care patient transfused total five units packed red blood cells given vitamin k counteract coagulopathy underwent egd banded sclerotherapy grade lower esophageal varices lower mid esophagus patient also abdominal ct followup chemoembo showed portal vein thrombosis present patient denied shortness breath chest pain headache nausea vomiting fever chills states feels much better past medical history significant hepatitis c virus alcoholic cirrhosis diagnosed duodenal 

In [58]:
df_train['TEXT'] = df_train['TEXT'].apply(str)

In [ ]:
df_valid['TEXT'] = df_valid['TEXT'].apply(str)

In [ ]:
# lambda apply clean_data function

In [59]:
df_train['TEXT'] = df_train.TEXT.apply(lambda x : clean_text(x))

In [60]:
df_valid['TEXT'] = df_valid.TEXT.apply(lambda x : clean_text(x))

In [61]:
# df_train.head()
df_valid.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE,DAYS_NEXT_ADMIT,CATEGORY,TEXT,OUTPUT_LABEL
25984,21284,126923,2168-05-03 07:15:00,2168-05-10 14:50:00,NaT,ELECTIVE,2171-02-04 04:04:00,EMERGENCY,999.551389,Discharge summary,f csu diagnosis aneurysm ascending aorta aorti...,0
11434,9344,116730,2199-05-21 02:59:00,2199-06-26 14:32:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,surgery allergies patient recorded known aller...,0
5215,4308,109424,2141-04-04 18:38:00,2141-04-06 16:52:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,medicine allergies patient recorded known alle...,0
53583,82935,157739,2183-01-17 20:44:00,2183-01-19 11:19:00,NaT,EMERGENCY,NaT,NaN,NaN,Discharge summary,f medicine allergies patient recorded known al...,0
33329,27266,165453,2197-10-17 15:27:00,2197-10-26 12:00:00,NaT,EMERGENCY,2197-11-24 19:46:00,EMERGENCY,29.323611,Discharge summary,f cardiothoracic allergies patient recorded kn...,1


In [62]:
X_train = df_train.loc [:, 'TEXT']
y_train = df_train.loc [:, 'OUTPUT_LABEL']
X_test = df_valid.loc [:, 'TEXT']
y_test = df_valid.loc [:, 'OUTPUT_LABEL']

In [ ]:
len(X_train[0])

5197

In [ ]:
# [k for k in range(1,2)]
X_train[0]
# sum(X_train.str.len())/4732

'history present illness patient year old male history hepatitis c virus hepatocellular cancer status post chemoembo presented hospital e hematemesis since one episode hematemesis two episodes well hematemesis hematochezia hospital e patient noted hematocrit transfused two units packed red blood cells noted coagulopathy inr given ffp well vitamin k patient denies past history varices upper gi bleed denies recent alcohol drug use feeling nauseated began vomiting continued hematemesis three days transferred hospital definitive care patient transfused total five units packed red blood cells given vitamin k counteract coagulopathy underwent egd banded sclerotherapy grade lower esophageal varices lower mid esophagus patient also abdominal ct followup chemoembo showed portal vein thrombosis present patient denied shortness breath chest pain headache nausea vomiting fever chills states feels much better past medical history significant hepatitis c virus alcoholic cirrhosis diagnosed duodenal 

In [63]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [64]:
max_leng = 1000
vocabulary_size = 5000
tokenizer = Tokenizer(num_words= vocabulary_size)
# tokenizer = Tokenizer(num_words=None, lower=True, split=" ")
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
len([sequences][0])

4832

In [65]:
# sequences[0]
X_train= pad_sequences(sequences, maxlen=max_leng)

In [ ]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [66]:
# tokenizer = Tokenizer(num_words=None, lower=True, split=" ")
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_test)
sequences = tokenizer.texts_to_sequences(X_test)

In [67]:
X_test= pad_sequences(sequences, maxlen=max_leng)

In [ ]:
X_test

array([[   0,    0,    0, ...,  416,  416,  158],
       [  12,  137,   21, ...,  428,  101,  303],
       [   0,    0,    0, ...,   56,  546,  212],
       ...,
       [   0,    0,    0, ...,   72,  158,  303],
       [ 123, 2832, 1073, ...,    7,  424, 1119],
       [ 190, 4174,  321, ...,   44, 2517,  233]], dtype=int32)

In [68]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, Flatten, MaxPooling1D, Reshape
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [70]:
model1 = Sequential()
model1.add(Embedding(5000,32,input_length=max_leng))
model1.add(Reshape((32, 1000)))
model1.add(LSTM(128))
model1.add(Dense(256, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC','accuracy'])
print(model1.summary())
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 32)          160000    
_________________________________________________________________
reshape_1 (Reshape)          (None, 32, 1000)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               578048    
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 771,329
Trainable params: 771,329
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
38/38 [==============================] - 3s 78ms/step - loss: 0.6797 - auc: 0.6001 - accuracy: 0.5

In [71]:
y_train_preds = model1.predict(X_train)
y_test_preds = model1.predict(X_test)

In [ ]:
#y_test_preds

In [73]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_preds)
fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_test, y_test_preds)


thresh = 0.5

auc_train1 = roc_auc_score(y_train, y_train_preds)
auc_test1 = roc_auc_score(y_test, y_test_preds)

In [74]:
auc_test1

0.5418639173365246

GloVe embedding

In [75]:
#https://nlp.stanford.edu/projects/glove/
embeddings_index = dict()
f = open('/content/drive/MyDrive/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [77]:
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.46751001,  0.34999999, -0.4745    , ..., -1.10839999,
         0.24685   , -0.30303001],
       [-0.44887   ,  1.02260005, -0.12047   , ..., -0.74816   ,
         0.89541   ,  0.35398   ],
       ...,
       [ 0.53767997, -0.33155999, -0.65117002, ...,  0.13197   ,
         0.59350002,  0.65812999],
       [ 0.24176   , -0.13823999,  0.13985001, ..., -0.020677  ,
        -1.11409998, -0.13709   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [81]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Conv1D, Flatten, MaxPooling1D, AveragePooling1D, Embedding, Dropout, BatchNormalization, Lambda
from keras.models import Sequential
import keras.backend as K
import keras

In [79]:
vocab_size

57253

In [82]:
model2 = Sequential()
# model2.add(Embedding(vocab_size, 100, input_length=max_leng, trainable=False))
model2.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_leng, trainable=False))
model2.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(None,100)))
model2.add(Reshape((100, 1)))
model2.add(Conv1D(4, 2, strides=1, padding='same', activation='relu'))
# # model2.add(BatchNormalization())
model2.add(Dropout(.5))
model2.add(MaxPooling1D())
model2.add(Flatten())
model2.add(Dense(1000, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
# sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
opt = keras.optimizers.Adam(learning_rate=0.0005)
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','AUC'])
model2.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy','AUC'])
model2.summary()
# Fit the model
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=128, verbose=2, shuffle=True)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1000, 100)         5725300   
_________________________________________________________________
lambda_1 (Lambda)            (None, 100)               0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 100, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 4)            12        
_________________________________________________________________
dropout_21 (Dropout)         (None, 100, 4)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 4)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 200)              

In [83]:
y_train_preds = model2.predict(X_train)
y_test_preds = model2.predict(X_test)
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_preds)
fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_test, y_test_preds)


thresh = 0.5

auc_train1 = roc_auc_score(y_train, y_train_preds)
auc_test1 = roc_auc_score(y_test, y_test_preds)
auc_test1

0.6428594374958079

BERTS

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 41.1MB/s 
     |████████████████████████████████| 983kB 41.2MB/s 
     |████████████████████████████████| 266kB 43.5MB/s 
     |████████████████████████████████| 1.3MB 42.9MB/s 
     |████████████████████████████████| 471kB 36.8MB/s 
     |████████████████████████████████| 1.1MB 34.2MB/s 
     |████████████████████████████████| 2.9MB 36.0MB/s 
     |████████████████████████████████| 890kB 41.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.0-cp36-none-any.whl size=25274500 sha256=36e26bcec2567c782fdf84396e7147bcd5b1faf7cb7bf7691f8b681a90b03618
  Stored in directory: /root/.cache/pip/wheels/5c/73/05/f36d0027bb6575384e21506dbba8db36a7825f15a24f09b2d5
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=3869d9f1bac3354b4278609f6ffd73a1c017a4e9b8160a23448a949e2591946a
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for la

In [ ]:
import ktrain
from ktrain import text
import tensorflow as tf

In [ ]:
(train, val, preproc) = text.texts_from_df(train_df=df_train, text_column='TEXT', label_columns='OUTPUT_LABEL',
                                           val_df = df_valid,
                                           maxlen = 512, #length of each doc aligned to 512
                                           preprocess_mode = 'distilbert')


preprocessing train...
language: en
train sequence lengths:
	mean : 1002
	95percentile : 2024
	99percentile : 2682


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 1431
	95percentile : 2947
	99percentile : 3971


In [ ]:
model4 = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 512



done.


In [ ]:
learner = ktrain.get_learner(model = model4,
                             train_data = train,
                             val_data = val,
                             batch_size = 8)

In [ ]:
learner.fit_onecycle(lr = 2e-4, epochs=2)



begin training using onecycle policy with max lr of 0.0002...
Epoch 1/2
604/604 [==============================] - 910s 2s/step - loss: 0.6980 - accuracy: 0.5029 - val_loss: 0.6747 - val_accuracy: 0.9425
Epoch 2/2
604/604 [==============================] - 907s 2s/step - loss: 0.6952 - accuracy: 0.4963 - val_loss: 0.6684 - val_accuracy: 0.9030


In [47]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [48]:
#X_test
train

In [84]:
X_test = df_valid.loc [:, 'TEXT']
# X_test.keys()
X_test_BERT = []
for keys in X_test.keys():
  X_test_BERT.append(X_test[keys])
X_test_BERT[0]

'f csu diagnosis aneurysm ascending aorta aortic arch hypertension hypercholesterolemia diabetes mellitus type hypothyroidism glaucoma osteoarthritis status post total abdominal hysterectomy status post colonic polypectomy diagnosis aneurysm ascending aorta aortic arch status post replacement ascending aorta hemi arch aortic valve resuspension mm gel weave tube graft hypertension hypercholesterolemia diabetes mellitus type hypothyroidism glaucoma osteoarthritis status post total abdominal hysterectomy status post colonic polypectomy history present illness patient year old female found mildly dilated abdominal aortic aneurysm work heart murmur diagnosed followed serially echocardiogram found grown cm noted otherwise asymptomatic given growing size aneurysm overall size scheduled elective repair physical examination vital signs patient afebrile hemodynamically normal pulse blood pressure systolic right arm systolic left arm saturating room air preoperative weight lb heart exam notable s

In [85]:
y_pred_BERT = predictor.predict(X_test_BERT, return_proba=True)

In [ ]:
#y_test 

20564    0
9061     0
4144     0
45720    1
26434    0
        ..
45999    0
28993    0
9555     0
47902    0
27269    1
Name: OUTPUT_LABEL, Length: 10223, dtype: int64

In [ ]:
predictor.save('/MyDrive/BERT_MIMIC_pred')

In [86]:
y_pred_BERT

array([[0.5158232 , 0.4841768 ],
       [0.51582015, 0.48417985],
       [0.5158181 , 0.48418182],
       ...,
       [0.5158211 , 0.4841789 ],
       [0.5158202 , 0.48417976],
       [0.51582104, 0.48417893]], dtype=float32)

In [87]:
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test, [np.argmax(y_pred_BERT[i]) for i in range(len(y_pred_BERT))])
print("\n ROC-AUC score: %.6f \n" % (score))


 ROC-AUC score: 0.482975 

